In [67]:
import pandas as pd
import numpy as np

df = pd.read_csv('npf_train.csv')


In [68]:
# Luodaan sarake class2 jossa on nonevent kun class4 == nonevent ja event kun class4 != nonevent

class2 = np.array(['event']*df.shape[0])
class2[df['class4']=='nonevent'] = 'nonevent'

df['class2'] = class2
df['class2'] = df['class2'].astype('category')

df.head()

,id,date,class4,partlybad,CO2168.mean,CO2168.std,CO2336.mean,CO2336.std,CO242.mean,CO242.std,...,T672.std,T84.mean,T84.std,UV_A.mean,UV_A.std,UV_B.mean,UV_B.std,CS.mean,CS.std,class2
0,1,2000-01-01,nonevent,False,384.462000,2.284996,384.164462,2.135062,385.274688,2.211695,...,0.525698,-12.422972,0.376324,1.635563,0.856948,0.026438,0.014617,0.003374,0.000733,nonev
1,2,2000-01-20,nonevent,False,374.884615,0.415185,374.703333,0.385179,375.621266,0.665720,...,0.373927,-8.351043,0.575679,1.441109,0.741088,0.022649,0.012479,0.001501,0.000572,nonev
2,3,2000-01-23,nonevent,False,373.496585,0.189497,373.382593,0.172958,373.961481,0.235107,...,0.965988,-9.651155,1.238891,2.677545,1.261612,0.044759,0.023748,0.000764,0.000048,nonev
3,4,2000-02-17,nonevent,False,378.600367,1.934180,378.464862,1.946536,379.785872,2.865022,...,0.122651,-0.829524,0.134191,2.261805,1.345651,0.030893,0.021903,0.002038,0.000751,nonev
4,5,2000-03-25,Ib,False,373.128684,1.096617,372.980000,1.047750,373.701830,1.259198,...,1.695622,-1.095864,2.090111,12.906779,7.022300,0.333523,0.239981,0.000662,0.000210,event


# Baseline malli
* Logistinen regressio
* https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
* ks. https://stackoverflow.com/questions/52670012/convergencewarning-liblinear-failed-to-converge-increase-the-number-of-iterati
* linkin takana vinkkejä 'failed to converge' varoitukseen
* https://scikit-learn.org/stable/modules/preprocessing.html#preprocessing-scaler

In [69]:
df.shape

(458, 105)

## Feature selection
- Select k-best (10 ~80% score)
https://scikit-learn.org/stable/modules/feature_selection.html#univariate-feature-selection

Variance threshold vois olla myös hyvä

In [70]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

from sklearn.model_selection import cross_val_score

X = df.drop(['id', 'date', 'class4', 'partlybad', 'class2'], axis=1).to_numpy()
y = df['class2'].to_numpy()

scaler = StandardScaler().fit(X)

X_scaled = scaler.transform(X) # mean=0, std_deviation=1, Standard Normal Distribution

X_new = SelectKBest(f_classif, k=80).fit_transform(X_scaled, y) # feature selection, 10 parasta

X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.33, random_state=42)

clf = LogisticRegression(random_state=0).fit(X_train, y_train)

#cross_val_predict(clf, X_new, y, cv=5)

score = clf.score(X_test, y_test)
print(f"score: {score}")

#clf.predict(X_test) #output järkevä

score: 0.8552631578947368


In [71]:
clf.predict_proba(X_test)

array([[9.64377453e-01, 3.56225473e-02],
       [7.55770741e-05, 9.99924423e-01],
       [9.01450495e-01, 9.85495053e-02],
       [8.26129549e-01, 1.73870451e-01],
       [9.99621648e-01, 3.78351712e-04],
       [5.78488491e-02, 9.42151151e-01],
       [1.71094496e-02, 9.82890550e-01],
       [9.57189026e-01, 4.28109743e-02],
       [9.25549138e-01, 7.44508623e-02],
       [8.22209542e-01, 1.77790458e-01],
       [2.76555900e-02, 9.72344410e-01],
       [9.98015650e-01, 1.98434999e-03],
       [2.99872391e-01, 7.00127609e-01],
       [3.16781544e-01, 6.83218456e-01],
       [2.12627707e-01, 7.87372293e-01],
       [9.74079499e-01, 2.59205009e-02],
       [1.15943751e-01, 8.84056249e-01],
       [7.13251346e-02, 9.28674865e-01],
       [1.09844434e-01, 8.90155566e-01],
       [3.30602504e-01, 6.69397496e-01],
       [2.02436992e-01, 7.97563008e-01],
       [9.38392177e-01, 6.16078228e-02],
       [7.08244896e-01, 2.91755104e-01],
       [7.54063953e-02, 9.24593605e-01],
       [1.547629

## Multilabel model
* Käyttää algoritmia 'newton-cg' -> tällä ei tullut convergence varoitusta

In [73]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import cross_val_score

df['class4'] = df['class4'].astype('category')
X2 = df.drop(['id', 'date', 'class4', 'partlybad', 'class2'], axis=1).to_numpy()
y2 = df['class4'].to_numpy()

scaler2 = StandardScaler().fit(X2)

X2_scaled = scaler2.transform(X2) # mean=0, std_deviation=1, Standard Normal Distribution

feature_selector = SelectKBest(f_classif, k=50).fit(X2_scaled, y2)
X2_new = feature_selector.transform(X2_scaled) # feature selection, 10 parasta

#X_train, X_test, y_train, y_test = train_test_split(X2_new, y2, test_size=0.33, random_state=42)

clf2 = LogisticRegression(random_state=0, solver='newton-cg', multi_class='multinomial') #.fit(X_train, y_train)
scores = cross_val_score(clf2, X2_new, y2, cv=5)

print(scores)
print(f"Cross val score mean: {scores.mean()}")
#labels = clf2.predict(X_test) #output järkevä


print(scores.mean())

[0.61956522 0.5326087  0.58695652 0.62637363 0.57142857]
Cross val score mean: 0.5873865265169613
0.5873865265169613


## Lue hidden-y data sisään
- treena clf2 koko datalla
- printtaa ennusteet

In [57]:
df_hidden = pd.read_csv('npf_test_hidden.csv')
X_hidden = df_hidden.drop(['id', 'date', 'class4', 'partlybad'], axis=1).to_numpy()
X_hidden.shape

(965, 100)

In [74]:
# treenaa classifier treenidatalla, jossa 50 saraketta
clf2.fit(X2_new, y2)

LogisticRegression(multi_class='multinomial', random_state=0,
                   solver='newton-cg')

In [76]:
X_hidden_scaled = scaler2.transform(X_hidden) # normaalijakaumalle
X_hidden_kbest = feature_selector.transform(X_hidden_scaled)
X_hidden_kbest.shape

(965, 50)

In [77]:
labels = clf2.predict(X_hidden_kbest)
labels

array(['nonevent', 'nonevent', 'II', 'II', 'nonevent', 'II', 'nonevent',
       'nonevent', 'II', 'nonevent', 'nonevent', 'II', 'nonevent',
       'nonevent', 'II', 'nonevent', 'II', 'nonevent', 'Ib', 'nonevent',
       'nonevent', 'nonevent', 'nonevent', 'nonevent', 'nonevent', 'II',
       'nonevent', 'II', 'nonevent', 'nonevent', 'nonevent', 'Ib',
       'nonevent', 'II', 'II', 'II', 'II', 'nonevent', 'II', 'nonevent',
       'nonevent', 'Ib', 'II', 'nonevent', 'II', 'nonevent', 'II', 'II',
       'II', 'II', 'Ib', 'nonevent', 'nonevent', 'II', 'nonevent',
       'nonevent', 'II', 'nonevent', 'II', 'nonevent', 'nonevent', 'II',
       'nonevent', 'Ib', 'II', 'nonevent', 'nonevent', 'II', 'II', 'Ib',
       'II', 'Ib', 'nonevent', 'nonevent', 'nonevent', 'II', 'nonevent',
       'II', 'nonevent', 'nonevent', 'II', 'II', 'nonevent', 'nonevent',
       'II', 'nonevent', 'nonevent', 'nonevent', 'nonevent', 'II',
       'nonevent', 'nonevent', 'nonevent', 'Ia', 'II', 'nonevent',
       '

In [78]:
probs = clf2.predict_proba(X_hidden_kbest)
probs[0]

array([0.23918157, 0.11723025, 0.11587104, 0.52771715])

In [80]:
with open('answers.csv', 'w') as f:
    print(f"{scores.mean()}\n", file=f)
    print(f"class4,p \n", file=f)
    for i in range(len(labels)):
        l = labels[i]
        maxp = max(probs[i])
        
        print(f"{l},{maxp}", file=f)